## **Resultado final - "Emergencias UNGRD"**
----
Este notebook integra los dos CSVs previos en uno solo. Se realizó un profundo proceso de limpieza en, aproximadamente, 18 fuentes de datos (Excel y JSON) que derivo en el CSV final que resulta del procesamiento de datos realizado en este notebook.

**Elaborado con base en**
* [​​​​​​​​​​​​Consolidado anual de emergencias ​​| UNGRD](https://portal.gestiondelriesgo.gov.co/Paginas/Consolidado-Atencion-de-Emergencias.aspx)
* [Emergencias UNGRD | Datos Abiertos Colombia](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Emergencias-UNGRD-/wwkg-r6te/about_data)

#### ***Importación de paquetes***

In [10]:
import pandas as pd

#### ***Cargar los CSVs mediante Pandas***

In [11]:
emergenciasConsolidados = pd.read_csv('../CSVs/EmergenciasCC(2003-2018, 2023)_RegionPacifico.csv')
emergenciasDatosAbiertos = pd.read_csv('../CSVs/EmergenciasCC(2019-2022)_RegionPacifico.csv')

#### ***Imprimir una muestra de los datos obtenidos***

##### **Obtenido desde los consolidados *(sin filtrar aún por evento)***

In [12]:
emergenciasConsolidados.sample(5)

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
40326,2023.0,2.0,2023-02-17,INCENDIO FORESTAL,BOSCONIA,CESAR,0,0,0,0,0,0,0,5.0
22940,2014.0,7.0,2014-07-21 00:00:00,VENDAVAL,SAMPUES,SUCRE,0,2,0,45,9,0,0,0.0
23944,2014.0,9.0,2014-09-29 00:00:00,INCENDIO FORESTAL,CORRALES,BOYACA,0,0,0,0,0,0,0,4.0
23597,2014.0,9.0,2014-09-01 00:00:00,INCENDIO ESTRUCTURAL,QUIMBAYA,QUINDIO,0,0,0,25,5,5,0,0.0
43249,2023.0,10.0,2023-10-02,INCENDIO FORESTAL,SOACHA,CUNDINAMARCA,0,0,0,0,0,0,0,2.0


##### **Obtenido desde el dataset de Datos Abiertos**

In [13]:
emergenciasDatosAbiertos.sample(5)

,AÑO,MES,FECHA,DEPARTAMENTO,MUNICIPIO,EVENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
3050,2022,4,2022-04-19,CAUCA,CALDONO,MOVIMIENTO EN MASA,0,0,0,488,122,0,1,0.0
1394,2021,12,2021-12-16,CAUCA,SUCRE,INUNDACION,0,0,0,0,0,0,0,0.0
2043,2022,10,2022-10-31,CAUCA,SUAREZ,VENDAVAL,0,0,0,8,2,0,0,0.0
2715,2022,11,2022-11-10,CAUCA,SUCRE,MOVIMIENTO EN MASA,0,0,0,376,223,6,13,0.0
1046,2021,3,2021-03-16,VALLE DEL CAUCA,GUADALAJARA DE BUGA,MOVIMIENTO EN MASA,0,0,0,0,0,0,1,0.0


#### ***Concatenar los dos dataframes***

In [14]:
# Lista de nombres de los dataframes
dataframes = [emergenciasConsolidados, emergenciasDatosAbiertos]

# Concatenar los dataframes en uno solo
emergencias = pd.concat(dataframes)

emergencias.sample(5)

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
26739,2015.0,8.0,2015-08-04 00:00:00,INUNDACION,CUMARIBO,VICHADA,0,0,0,2190,550,0,0,0.0
36521,2018.0,4.0,2018-04-01,INCENDIO FORESTAL,UNGUIA,CHOCO,0,0,0,0,0,0,0,0.0
10365,2011.0,4.0,2011-04-21 00:00:00,DESLIZAMIENTO,CHINACOTA,NORTE DE SANTANDER,0,0,0,312,78,0,0,0.0
9593,2010.0,12.0,2010-12-31 00:00:00,INUNDACION,IQUIRA,HUILA,0,0,0,80,27,0,0,0.0
27073,2015.0,8.0,2015-08-26 00:00:00,INCENDIO FORESTAL,PALERMO,HUILA,0,0,0,0,0,0,0,20.0


## **Limpieza de datos**
---

### ***Consolidados - UNGRD***

#### ***Escogemos las filas con los eventos de nuestro interés***
En especial aquellos que son impactados fuertemente por el cambio climático.

In [15]:
emergencias["EVENTO"] = emergencias["EVENTO"].fillna("")

emergencias["EVENTO"] = emergencias["EVENTO"].str.upper()

emergencias["EVENTO"] = emergencias["EVENTO"].str.strip()

# Diccionario de reemplazo
diccionario_reemplazos = {
    "INUNDACION": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "INUNDACIÓN": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "AVENIDA TORRENCIAL": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE SUBITA": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE SÚBITA": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "LLUVIAS": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "VENDAVAL-INCENDIO ESTRUCTURAL": "VENDAVAL",
    "DELIZAMIENTO": "DESLIZAMIENTO",
    "MOVIMIENTO EN MASA": "DESLIZAMIENTO",
    "REMOCIÓN EN MASA": "DESLIZAMIENTO",
    "REMOCION EN MASA": "DESLIZAMIENTO",
    "MAREJADA": "MAREJADAS/MAR DE LEVA",
    "MAREJADAS": "MAREJADAS/MAR DE LEVA",
    "MAR DE LEVA": "MAREJADAS/MAR DE LEVA",
    "MAR DELEVA": "MAREJADAS/MAR DE LEVA",
    "EROSION-INCENDIO ESTRUCTURAL": "EROSION",
    "EROSION COSTERA": "EROSION",
    "FENOMENO ATMOSFERICO": "TEMPORAL",
    "CICLON TROPICAL: DEPRESION/TORMENTA/HURACAN": "DEPRESION/TORMENTA/HURACAN",
    "CICLÓN, TROPICAL, DEPRESIÓN/TORMENTA/HURACÁN": "DEPRESION/TORMENTA/HURACAN",
    "TORMENTA ELECTRICA": "DEPRESION/TORMENTA/HURACAN",
    "TORNADO": "DEPRESION/TORMENTA/HURACAN",
    "DEPRESION TROPICAL": "DEPRESION/TORMENTA/HURACAN",
    "HURACAN": "DEPRESION/TORMENTA/HURACAN",
}

# Aplicar los cambios
emergencias["EVENTO"] = emergencias["EVENTO"].replace(diccionario_reemplazos)

In [16]:
# Lista de eventos a filtrar (ajustada a los nombres proporcionados)
eventos_a_filtrar = [
    "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "INCENDIO FORESTAL",
    "DESLIZAMIENTO",
    "VENDAVAL",
    "SEQUIA",
    "DEPRESION/TORMENTA/HURACAN",
    "GRANIZADA",
    "TEMPORAL",
    "EROSION",
    "HELADA",
    "MAREJADAS/MAR DE LEVA"
]

# Filtrar el DataFrame
emergencias = emergencias[emergencias["EVENTO"].isin(eventos_a_filtrar)]

# Mostrar el DataFrame filtrado
emergencias

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
0,2003.0,1.0,2003-01-01,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,BAGADO,CHOCO,1,0,0,700,175,0,0,0.0
1,2003.0,1.0,2003-01-01,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,MEDIO ATRATO,CHOCO,0,0,0,3161,691,0,0,0.0
2,2003.0,1.0,2003-01-01,VENDAVAL,LITORAL DEL SAN JUAN,CHOCO,0,0,0,995,199,0,0,0.0
3,2003.0,1.0,2003-01-03,INCENDIO FORESTAL,LA TEBAIDA,QUINDIO,0,0,0,0,0,0,0,0.0
4,2003.0,1.0,2003-01-03,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,LLORO,CHOCO,0,0,0,1076,269,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3544,2022.0,12.0,2022-12-26,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,ROSAS,CAUCA,0,0,0,0,0,0,0,0.0
3545,2022.0,12.0,2022-12-26,DESLIZAMIENTO,ROSAS,CAUCA,0,0,0,7,2,0,0,0.0
3546,2022.0,12.0,2022-12-26,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,PADILLA,CAUCA,0,0,0,0,0,0,0,0.0
3572,2022.0,11.0,2022-11-25,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,DEPARTAMENTO,VALLE DEL CAUCA,0,0,0,0,0,0,0,0.0


#### ***Arregar formato en la fecha***

In [17]:
emergencias = emergencias.dropna()
emergencias = emergencias[emergencias != '']

if emergencias is not None:
    emergencias['AÑO'] = emergencias['AÑO'].astype(int, errors='ignore')
    emergencias['MES'] = emergencias['MES'].astype(int, errors='ignore')

# Eliminar la hora de la columna de fecha
emergencias['FECHA'] = pd.to_datetime(emergencias['FECHA'], format='ISO8601')
emergencias['FECHA'] = emergencias['FECHA'].dt.date

# Organizar los años y los meses de manera ascendente
emergencias = emergencias.sort_values(by=['AÑO', 'MES'], ascending=[True, True])

### ***Guardar como CSV***
---

In [18]:
emergencias.to_csv('../EmergenciasCC(2003-2023)_RegionPacifico.csv', index=False)